# Fixing the seeds 

In [ ]:
import torch
import random
import numpy as np

def set_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if you are using multi-GPU.
    np.random.seed(seed_value)  # Numpy module.
    random.seed(seed_value)  # Python random module.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True



set_seed(42)

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Dataset

In [ ]:
# from Experiment1.models import *
import os

os.chdir('../')
cwd = os.getcwd()
print(cwd)

/home/mohammad/mohamed-thesis


In [ ]:
from CUB.dataset import load_data
# def load_data(pkl_paths, use_attr, no_img, batch_size, uncertain_label=False, n_class_attr=2, image_dir='images', resampling=False, resol=299):
n_concepts = 112
batch_size = 16
test_loader = load_data(['CUB_processed/class_attr_data_10/test.pkl'], True, False, 32)
train_loader = load_data(['CUB_processed/class_attr_data_10/train.pkl'], True, False, 32)

# Models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Sequential(nn.Module):
    def __init__(self, model1_path, model2_path):
        super(Sequential, self).__init__()
        self.model1 = torch.load(model1_path)
        self.model2 = torch.load(model2_path)

    def forward(self, x):
        # try:
        #     x = self.model1(x)
        #     x = torch.stack(x).squeeze().t()
        #     x = self.model2(x)
        # except:
        #     x = torch.stack(x[0]).squeeze().t()
        #     x = self.model2(x)
        # print(x.shape)
        x = self.model1(x)
        # print(x.shape)
        x = torch.stack(x).squeeze().t()
        # print(x)
        x = self.model2(x)
        # print(x.shape)
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        return x



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Joint(nn.Module):
    def __init__(self, model_path):
        super(Joint, self).__init__()
        self.model = torch.load(model_path)

    def forward(self, x):
        x = self.model(x)[0]
        if isinstance(x, list):
            x = x[0]
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Standard(nn.Module):
    def __init__(self, model_path):
        super(Standard, self).__init__()
        self.model = torch.load(model_path)

    def forward(self, x):
        x = self.model(x)[0]
        if isinstance(x, list):
            x = x[0]
        return x

# Loading the models

In [ ]:
model_1_path = r'ConceptModel__Seed1 old/output/best_model_1.pth'
model_2_path = r'SequentialModel_WithVal__Seed1/best_model_42.pth'
sequential = Sequential(model_1_path, model_2_path).to(device)
sequential.eval()
print('done')

done


In [ ]:
import torch

joint_model_path = 'Joint0.01Model__Seed1/outputs/best_model_1.pth'
joint_model = Joint(joint_model_path).to(device)
joint_model.eval()
print(1)

1


In [ ]:
import torch

standard_model = Standard("Standard0Model_Seed1/outputs/best_model_1.pth").to(device)
standard_model.eval()
print(1)

1


# Attacking the models with other attacks

In [ ]:
!pip install foolbox

In [ ]:
import pandas as pd
def deepfool_attack(model, test_loader, eps):
    bounds = (-0.25, 0.25)
    fmodel = fb.PyTorchModel(model, bounds=bounds)
    # Set the attack method
    attack = fb.attacks.LinfDeepFoolAttack()

    # Track the number of correct predictions.
    correct = 0
    total = 0

    # Wrap the test_loader with tqdm for a progress bar
    for images, labels, _ in tqdm(test_loader, desc="Attacking Images"):
        images, labels = images.to(device), labels.to(device)

        # Run the attack
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
        
        # Predict the labels for the adversarial images
        logits = model(clipped_advs)
        preds = torch.argmax(logits, dim=1)
        
        # Update the number of correct predictions.
        correct += torch.sum(preds == labels).item()
        total += len(labels)

    # Calculate the final accuracy for the adversarial images.
    final_accuracy = correct / total
    print(f"Accuracy on adversarial examples: {final_accuracy*100:.2f}%")
    return final_accuracy

models = ['Sequential', 'joint_model', 'standard_model']
models_classes = {'Sequential': Sequential, 'joint_model': Joint, 'standard_model': Standard}
save_models_path = r"src/Adversarially trained models"
csv_file_path = r"src/Adversarially trained models/experiments.csv"
attack_config = {
    'eps_step': 0.01,
    'max_iter': 10,
    'clip_values': (-0.25, 0.25),
    'input_shape': (3, 299, 299),
    'nb_classes': 200
}


eps_values = [0.0005, 0.001, 0.0025, 0.005]
experiments = []
# Loading the models
for model_name in models:
  for eps in eps_values:
    experiment = {
        'model_name': model_name,
        'Trained on eps': eps
    }
    if model_name == 'Sequential':
        model_1_path = r'ConceptModel__Seed1 old/output/best_model_1.pth'
        model_2_path = r'SequentialModel_WithVal__Seed1/best_model_42.pth'
        model = Sequential(model_1_path, model_2_path).to(device)
        model.eval()
    elif model_name == 'joint_model':
        joint_model_path = 'Joint0.01Model__Seed1/outputs/best_model_1.pth'
        model = Joint(joint_model_path).to(device)
        model.eval()
    elif model_name == 'standard_model':
        model = Standard("Standard0Model_Seed1/outputs/best_model_1.pth").to(device)
        
    model_path = f"{save_models_path}/{model_name}_{str(eps).split('.')[1]}.pth"
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded {model_name}_{str(eps).split('.')[1]}")
    # selected_attack = attack_mapping['CWL2']
    model.eval()
    # accuracy = attack_deepfool(model, test_loader,test_labels, attack_config)
    accuracy = deepfool_attack(model, test_loader, eps)
    print(accuracy)



Loaded Sequential_0005


Attacking Images: 100%|██████████| 182/182 [22:05<00:00,  7.28s/it]


Accuracy on adversarial examples: 43.61%
0.4361408353469106
Loaded Sequential_001


Attacking Images: 100%|██████████| 182/182 [22:57<00:00,  7.57s/it]


Accuracy on adversarial examples: 37.09%
0.3709009319986193
Loaded Sequential_0025


...

Accuracy on adversarial examples: 43.46%
0.4345875043148084
Loaded standard_model_0025


Attacking Images: 100%|██████████| 182/182 [23:40<00:00,  7.80s/it]


Accuracy on adversarial examples: 35.28%
0.35277873662409387
Loaded standard_model_005


Attacking Images: 100%|██████████| 182/182 [24:33<00:00,  8.09s/it]

Accuracy on adversarial examples: 24.66%
0.2466344494304453


In [ ]:
import foolbox as fb
from tqdm import tqdm

def cw_l2_attack(model, test_loader, eps):
    bounds = (-0.25, 0.25)
    fmodel = fb.PyTorchModel(model, bounds=bounds)
    # Set the attack method
    attack = fb.attacks.L2CarliniWagnerAttack(steps=10, confidence=0)

    # Track the number of correct predictions.
    correct = 0
    total = 0

    # Wrap the test_loader with tqdm for a progress bar
    for images, labels, _ in tqdm(test_loader, desc="Attacking Images"):
        images, labels = images.to(device), labels.to(device)

        # Run the attack
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
        
        # Predict the labels for the adversarial images
        logits = model(clipped_advs)
        preds = torch.argmax(logits, dim=1)
        
        # Update the number of correct predictions.
        correct += torch.sum(preds == labels).item()
        total += len(labels)

    # Calculate the final accuracy for the adversarial images.
    final_accuracy = correct / total
    print(f"Accuracy on adversarial examples: {final_accuracy*100:.2f}%")
    return final_accuracy

import pandas as pd

models = ['Sequential', 'joint_model', 'standard_model']
models_classes = {'Sequential': Sequential, 'joint_model': Joint, 'standard_model': Standard}
save_models_path = r"src/Adversarially trained models"
csv_file_path = r"src/Adversarially trained models/experiments.csv"
attack_config = {
    'eps_step': 0.01,
    'max_iter': 10,
    'clip_values': (-0.25, 0.25),
    'input_shape': (3, 299, 299),
    'nb_classes': 200
}
# # Example usage:
# attack_mapping = {
#     'CWL2': CarliniL2Method,
#     'CWLinf': CarliniLInfMethod,
#     'DeepFool': DeepFool
# }

eps_values = [0.0005, 0.001, 0.0025, 0.005]
experiments = []
# Loading the models
for model_name in models:
  for eps in eps_values:
    experiment = {
        'model_name': model_name,
        'Trained on eps': eps
    }
    if model_name == 'Sequential':
        model_1_path = r'ConceptModel__Seed1 old/output/best_model_1.pth'
        model_2_path = r'SequentialModel_WithVal__Seed1/best_model_42.pth'
        model = Sequential(model_1_path, model_2_path).to(device)
        model.eval()
    elif model_name == 'joint_model':
        joint_model_path = 'Joint0.01Model__Seed1/outputs/best_model_1.pth'
        model = Joint(joint_model_path).to(device)
        model.eval()
    elif model_name == 'standard_model':
        model = Standard("Standard0Model_Seed1/outputs/best_model_1.pth").to(device)
        
    model_path = f"{save_models_path}/{model_name}_{str(eps).split('.')[1]}.pth"
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded {model_name}_{str(eps).split('.')[1]}")
    # selected_attack = attack_mapping['CWL2']
    model.eval()
    # accuracy = attack_deepfool(model, test_loader,test_labels, attack_config)
    accuracy = cw_l2_attack(model, test_loader, eps)
    print(accuracy)

Loaded Sequential_0005


Attacking Images: 100%|██████████| 182/182 [31:14<00:00, 10.30s/it]


Accuracy on adversarial examples: 58.51%
0.5850880220918191
Loaded Sequential_001


Attacking Images: 100%|██████████| 182/182 [35:05<00:00, 11.57s/it]


Accuracy on adversarial examples: 55.13%
0.5512599240593717
Loaded Sequential_0025


...

Attacking Images: 100%|██████████| 182/182 [31:13<00:00, 10.29s/it]


Accuracy on adversarial examples: 61.74%
0.6173627890921644
Loaded standard_model_001


Attacking Images: 100%|██████████| 182/182 [36:25<00:00, 12.01s/it]


Accuracy on adversarial examples: 62.15%
0.6215050051777701
Loaded standard_model_0025


Attacking Images:  54%|█████▍    | 98/182 [25:28<23:03, 16.47s/it]

In [ ]:
import foolbox as fb
from tqdm import tqdm

def cw_l2_attack(model, test_loader, eps):
    bounds = (-0.25, 0.25)
    fmodel = fb.PyTorchModel(model, bounds=bounds)
    # Set the attack method
    attack = fb.attacks.L2CarliniWagnerAttack(steps=10, confidence=0)

    # Track the number of correct predictions.
    correct = 0
    total = 0

    # Wrap the test_loader with tqdm for a progress bar
    for images, labels, _ in tqdm(test_loader, desc="Attacking Images"):
        images, labels = images.to(device), labels.to(device)

        # Run the attack
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=eps)
        
        # Predict the labels for the adversarial images
        logits = model(clipped_advs)
        preds = torch.argmax(logits, dim=1)
        
        # Update the number of correct predictions.
        correct += torch.sum(preds == labels).item()
        total += len(labels)

    # Calculate the final accuracy for the adversarial images.
    final_accuracy = correct / total
    print(f"Accuracy on adversarial examples: {final_accuracy*100:.2f}%")
    return final_accuracy

import pandas as pd

models = ['Sequential', 'joint_model', 'standard_model']
models_classes = {'Sequential': Sequential, 'joint_model': Joint, 'standard_model': Standard}
save_models_path = r"src/Adversarially trained models"
csv_file_path = r"src/Adversarially trained models/experiments.csv"
attack_config = {
    'eps_step': 0.01,
    'max_iter': 10,
    'clip_values': (-0.25, 0.25),
    'input_shape': (3, 299, 299),
    'nb_classes': 200
}
# # Example usage:
# attack_mapping = {
#     'CWL2': CarliniL2Method,
#     'CWLinf': CarliniLInfMethod,
#     'DeepFool': DeepFool
# }

eps_values = [0.0005, 0.001, 0.0025, 0.005]
experiments = []
# Loading the models
for model_name in models:
  if model_name != 'standard_model':
     continue
  for eps in eps_values[2:]:
    experiment = {
        'model_name': model_name,
        'Trained on eps': eps
    }
    if model_name == 'Sequential':
        model_1_path = r'ConceptModel__Seed1 old/output/best_model_1.pth'
        model_2_path = r'SequentialModel_WithVal__Seed1/best_model_42.pth'
        model = Sequential(model_1_path, model_2_path).to(device)
        model.eval()
    elif model_name == 'joint_model':
        joint_model_path = 'Joint0.01Model__Seed1/outputs/best_model_1.pth'
        model = Joint(joint_model_path).to(device)
        model.eval()
    elif model_name == 'standard_model':
        model = Standard("Standard0Model_Seed1/outputs/best_model_1.pth").to(device)
        
    model_path = f"{save_models_path}/{model_name}_{str(eps).split('.')[1]}.pth"
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded {model_name}_{str(eps).split('.')[1]}")
    # selected_attack = attack_mapping['CWL2']
    model.eval()
    # accuracy = attack_deepfool(model, test_loader,test_labels, attack_config)
    accuracy = cw_l2_attack(model, test_loader, eps)
    print(accuracy)

Loaded standard_model_0025


Attacking Images: 100%|██████████| 182/182 [44:46<00:00, 14.76s/it]


Accuracy on adversarial examples: 61.65%
0.6164998274076631
Loaded standard_model_005


Attacking Images: 100%|██████████| 182/182 [47:14<00:00, 15.58s/it]

Accuracy on adversarial examples: 58.73%
0.5873317224715222
